# Introduction to Python sockets

Sockets are a communication link between two different endpoints.

In networks, each endpoint is usually an address (specified at the Transport and Internet layer).

In a client-server architecture, these sockets have to be managed by the client and server software.

The software usually cannot create a socket by itself; it has to request one through the operating system.

# Using sockets in Python

In Python, the `socket` module provides functions for managing sockets.

Let’s create a simple socket:

In [ ]:
import socket

# Let's create an unbound socket
# The socket() function requires two arguments:
# 1. the address family
#    ('AF_INET' for IPv4 or 'AF_INET6' forIPv6)
# 2. the socket type
#    ('SOCK_STREAM' for TCP or 'SOCK_DGRAM' for UDP)
s = socket.socket(
    socket.AF_INET,     # address family: IPv4
    socket.SOCK_STREAM, # socket type: TCP
)

# sockets must be closed after use
s.close()

This gives us a socket, one end of which is used by Python.

This socket doesn’t do anything yet.

For it to work, we have to do something with the other end.

For a *server* (not covered in this lesson), we have to **bind** the other end to another hostname for listening.

For a *client*, we have to **connect** the other end to another address.

## IPv4 socket addresses

An address specifies a destination that data will be sent to, and which a response will be received from.

For an IPv4 connection, this is specified by a hostname and a port number, as a 2-element tuple.

In [ ]:
# The hostname can be a domain or IP address.
# Here, we are connecting to a server running on the same computer
# (Make sure server.py is running first)
HOST = '127.0.0.1'

# Only non-reserved ports (49152-65535) should be used
# for nonstandard protocols used in custom apps
PORT = 65535

ADDR = (HOST, PORT)

## Securing sockets

Unclosed sockets are a security vulnerability; other software/users that know the address can attempt to send data to it!

If a socket is opened, and not properly closed (e.g. if the program crashes), it remains active and available for data transfer.

A safe way to ensure that sockets are always closed properly is to establish them using the `with` keyword (similar to file IO):

In [ ]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect(ADDR)
    print(f"Connection to {ADDR} is open!")
print(f"Connection to {ADDR} is closed.")

## Sending a request

In a client-server model, the client always initiates the request. Thus, after you connect to the server, nothing happens until the client sends a request.

When the client is sending data to the server, the server has no way of knowing when the data has been completely sent. The client may have crashed halfway, or the connection may have been temporarily interrupted during sending.

The only way to know for sure is to establish a communication protocol beforehand (this forms the Application layer).

A common protocol is to include the length of the message at the start (e.g. in the first 1-2 bytes), so that the server can read it and know how much more data to expect.

Here, we will simply assume that each data packet is no more than 1024 bytes.

Let’s send some data to the server. We do so using the `send()` method of the *socket object*.

In [ ]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect(ADDR)
    s.send('hi')

`TypeError: a bytes-like object is required, not 'str'`

This happens because a socket only sends bytes. Other data types—`int`, `str`, `float`, etc—must be converted to `bytes` ([another data type in Python](https://docs.python.org/3.6/c-api/bytes.html)) before they can be passed to `send()`.

Fortunately, `str`-type objects have a built-in method to do so.

Calling the `str.encode()` method returns a value as `bytes`. By default, the `str` is assumed to be encoded in `utf-8` encoding unless otherwise specified.

In [ ]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect(ADDR)
    s.send('hi'.encode())

## Receiving a response

Hmm ... nothing happened.

Actually, the server should have returned a response (if it up and running). But we have to *receive* the data from the socket to know what the response is.

We do so using the `recv()` method of the socket object. This method requires you to specify how many bytes to read from the received data.

In [ ]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect(ADDR)
    s.send('hi'.encode())
    data = s.recv(1024)  # Read up to 1024 bytes from the response
    print(data)

`server.py` is a simple server that just echoes back the data you sent to it, so it is easy to tell if you have done it correctly; you should see the same data returned!

## `bytes` data type

Notice that the response is not `'hi'`, but `b'hi'`.

The `b` in front indicates that this value is a `bytes` object, not a `str` object:

In [ ]:
type(data)

Before we can concatenate it to other strings or use string methods on it, we have to convert it to a `str` object first.

`bytes`-type objects have a built-in method, `decode()`, to do so. This will return a `utf-8`-encoded `str` unless a different encoding is specified.

In [ ]:
str_data = data.decode()
print(str_data)

We have successfully sent a request to a server, and read its response!

# A request-response loop

Our program only completed one request-response loop before terminating.

In practical applications, the request-response loop often needs to happen multiple times. This means we usually need to do so in a `while` loop, until a terminating condition is met.

Here is what a full program that sends requests and receives responses in a loop might look like:

In [ ]:
import socket

ADDR = ('127.0.0.1', 65535)

exit = False
while not exit:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect(ADDR)
        msg = input("Say something ('exit' to quit): ")
        s.send(msg.encode())
        if msg == 'exit':
            exit = True
        else:
            resp = s.recv(1024)
            print("Response:", resp.decode())

# Sustained connections

In the above example, each iteration of the loop opens a new connection before sending data.

This is acceptable for loops that don’t repeat too quickly. Requesting a new connection takes time and OS resources, and doing it too often can lead to system slowdown.

In cases where requests need to be sent and responses received very frequently, the socket may be left open (i.e. `while` loop occurs within the `with` statement) while data exchange is going on, and only closed after all data exchange is complete.

However, note that this will take up resources on the server, which has to keep the connection open. Most computer OSes are configured with an upper limit of 128 concurrent connections at any point (unless this setting is overridden).

It is considered polite to close a connection and free up this resource when the client no longer needs it.